In [1]:
# !pip3 install metpy
# !pip3 install pathlib # instal if needed

# not work when connect to PNNL VPN

In [195]:
# Correction Factor
def calculate_Factor(bf_read, factors):
    bf05, bf10, bf20 = factors[0], factors[1], factors[2]
    if bf_read == 0.05:
        return bf05
    elif bf_read == 0.10:
        return bf10
    elif bf_read == 0.20:
        return bf20
    elif bf_read < 0.05:
        return bf05 - ((0.05 - bf_read) * (bf10 - bf05) / 0.05)
    elif bf_read > 0.05 and bf_read < 0.10:
        return bf05 + ((bf_read - 0.05) * (bf10 - bf05) / 0.05)
    elif bf_read > 0.10 and bf_read < 0.20:
        return bf10 + ((bf_read - 0.10) * (bf20 - bf10) / 0.1)
    else:
        return bf20 + ((bf_read - 0.2) * (bf20 - bf10) / 0.1)

# 48-50A-21PD-SM
def correctionFactor_1(row):
    factors = []
    # get correction factor
    if row['EADB'] in [79, 81]:
        factors = [1.04, 0.98, 0.87]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [78, 82]:
        factors = [2.07, 1.96, 1.74]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [77, 83]:
        factors = [3.11, 2.94, 2.62]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [76, 84]:
        factors = [4.14, 3.92, 3.49]
        factor = calculate_Factor(row['BF'], factors)
    elif row['EADB'] in [75, 85]:
        factors = [4.18, 4.91, 4.36]
        factor = calculate_Factor(row['BF'], factors)
    else:
        factor = 1.1 * (1 - row['BF']) * (row['EADB'] - 80)
    
    modified_row = row.copy()
        
    # modify 'SHC'
    if row['EADB'] < 80:
        modified_row['SHC'] = row['SHC'] - factor * row['airflow']
    elif row['EADB'] > 80:
        modified_row['SHC'] = row['SHC'] + factor * row['airflow']
    return modified_row


In [273]:
import pandas as pd
import numpy as np
import metpy.calc as mcalc
from metpy.units import units
from pathlib import Path

def loadData(name,cols,nHead,nRows):
    ## Load the data from a page in the excel file
    df = pd.read_excel(str(name), sheet_name='Sheet1', header=int(nHead), 
                       usecols=cols, skiprows=None, nrows=int(nRows), engine="openpyxl")
    df = df.replace('—', pd.NA)
    
    ## Rotate data
    column_index_to_drop = [0]
    rotate_df = df.drop(df.columns[column_index_to_drop], axis=1)
    buffer_df = rotate_df.T
    buffer_df.reset_index(inplace=True)
    buffer_df.columns = buffer_df.iloc[0]
    buffer_df = buffer_df.iloc[1:]
    
    ## Read EAWB
    eawb_values = buffer_df.iloc[:5, 0]
    eawb_values = eawb_values.tolist()
    eawb_times = len(buffer_df)//5
    eawb_list = eawb_values * eawb_times
    buffer_df.rename(columns={'Unnamed: 1':'EAWB'}, inplace=True)
    buffer_df['EAWB'] = eawb_list

    
    ## Read cfm
    cfm = pd.read_excel(str(name), sheet_name='Sheet1', header=int(nHead-3), 
                       usecols=cols, skiprows=None, nrows=1, engine="openpyxl")
    cfm_values = np.array(cfm)[0]
    cfm_values = cfm_values.tolist()
    cfm_list = [int(value) for value in cfm_values if not pd.isna(value)]
    cfm_array = np.repeat(cfm_list,5).tolist()
    buffer_df.insert(0, "airflow", np.nan)
    buffer_df['airflow'] = cfm_array
    
    ## Modify the dataframe
    num_chunk = len(buffer_df.columns)//4
    if num_chunk <= 5:
        chunk1 = buffer_df.iloc[:, :2]
        chunk2 = buffer_df.iloc[:, 2:6]
        chunk3 = buffer_df.iloc[:, 6:10]
        chunk4 = buffer_df.iloc[:, 10:14]
        chunk5 = buffer_df.iloc[:, 14:18]
        chunk6 = buffer_df.iloc[:, 18:22]
        result_df = pd.concat([chunk2, chunk3, chunk4, chunk5, chunk6], ignore_index=True)
        chunk1_df = pd.concat([chunk1, chunk1, chunk1, chunk1, chunk1], ignore_index=True)
        chunk1_df.reset_index(inplace=True)
    else:
        chunk1 = buffer_df.iloc[:, :2]
        chunk2 = buffer_df.iloc[:, 2:6]
        chunk3 = buffer_df.iloc[:, 6:10]
        chunk4 = buffer_df.iloc[:, 10:14]
        chunk5 = buffer_df.iloc[:, 14:18]
        chunk6 = buffer_df.iloc[:, 18:22]
        chunk7 = buffer_df.iloc[:, 22:26]
        result_df = pd.concat([chunk2, chunk3, chunk4, chunk5, chunk6, chunk7], ignore_index=True)
        chunk1_df = pd.concat([chunk1, chunk1, chunk1, chunk1, chunk1, chunk1], ignore_index=True)
        chunk1_df.reset_index(inplace=True)   
    merged_df = pd.concat([chunk1_df, result_df], axis=1)
    merged_df = merged_df.iloc[:, 1:]
    
    ## Read EADB
    eadb_values = df.iloc[:, 0].tolist()
    eadb_values = [int(value) for value in eadb_values if not pd.isna(value)]
    eadb_times = 5 * len(cfm_list)
    eadb_array = np.repeat(eadb_values,eadb_times).tolist()
    merged_df['EADB'] = eadb_array
    merged_df['OADB'] = eadb_array
    
    ## Clean NA
    
    
    ## Calculate RH
    # The relative_humidity_wet_psychrometric function requires temperature and wet bulb temperature in Kelvin,
    # And pressure in Pascals, we assume standard pressure at sea level: 101325 Pa
    pressure = 101325 * units.pascal
    # Convert Celsius to Kelvin and compute relative humidity
    merged_df['RH'] = merged_df.apply(lambda row: mcalc.relative_humidity_wet_psychrometric(pressure,row['EADB'] * units.degF, row['EAWB'] * units.degF).to('percent').magnitude, axis=1)
    
    ## Modify units
    merged_df['TC'] = merged_df['TC']*1000
    merged_df['SHC'] = merged_df['SHC']*1000
    
    ## Supply fan power
    merged_df.insert(0, "SFP", np.nan)
    
    
    ## Correct SHC
    merged_df = merged_df.apply(lambda row: correctionFactor_1(row), axis=1)
    
    ## organize
#     # for test
#     merged_df['S2T'] = merged_df['SHC']/merged_df['TC']
#     new_cols = ['OADB', 'EADB', 'EAWB', 'RH', 'TC', 'SHC', 'S2T', 'kW', 'airflow']

    new_cols = ['OADB', 'EADB', 'EAWB', 'RH', 'TC', 'SHC', 'kW', 'airflow', 'SFP']
    final = merged_df[new_cols]
    
    return final

In [274]:
# 48-50A-21PD-SM
column_names = ['OADB', 'EADB', 'EAWB', 'RH', 'TC', 'SHC', 'kW', 'airflow', 'SFP']
gap = pd.DataFrame(columns=column_names)
new_row = pd.Series(['-------------'] * len(column_names), index=column_names)
gap = gap.append(new_row, ignore_index=True)


# 020
test020_1 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:V",5,24)
test020_2 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:Q",35,24)
test020 = pd.concat([test020_1,test020_2,gap], axis=0)
# print(test020)

# 025
test025_1 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:V",65,20)
test025_2 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:Q",91,20)
test025 = pd.concat([test025_1,test025_2,gap], axis=0)

# 027
test027_1 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:V",117,20)
test027_2 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:Q",143,20)
test027 = pd.concat([test027_1,test027_2,gap], axis=0)

# 030
test030_1 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:V",169,20)
test030_2 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:Q",195,20)
test030 = pd.concat([test030_1,test030_2,gap], axis=0)

# 035
test035_1 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:V",221,20)
# test035_2 = loadData('./excel/48-50A-21PD-SM.xlsx',"A:Q",246,20)
# test035 = pd.concat([test035_1,test035_2], axis=0)
# # drop NA
# test035 = test035.dropna()
# test035 = test035.reset_index(drop=True)

# 040

# 050

# 060

final = pd.concat([test020,test025,test027,test030], axis=0)
# print(final)
file_path = Path('./output/48-50A-21PD.csv')
final.to_csv(file_path, index=False)

C:\Users\zhan682\AppData\Local\Temp\ipykernel_17244\2071166645.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gap = gap.append(new_row, ignore_index=True)
C:\Users\zhan682\AppData\Local\Temp\ipykernel_17244\653083705.py:31: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  cfm = pd.read_excel(str(name), sheet_name='Sheet1', header=int(nHead-3),
